<a href="https://colab.research.google.com/github/wissal-mahri/data-science-colab-projects/blob/main/GreenShop_DATAcleaning_DW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CLIENT**

In [ ]:
import pandas as pd
import re

# Lien de partage fourni par l'utilisateur (sera transformé)
share_link = 'https://docs.google.com/spreadsheets/d/1SSAlS1aGriDgKKu2AaCGksi4ThEnLBaNNcMklx5dAJU/edit?usp=sharing'

# --- TRANSFORMATION AUTOMATIQUE DU LIEN ---
match = re.search(r'/d/([a-zA-Z0-9_-]+)', share_link)

if match:
    spreadsheet_id = match.group(1)
    # Construction du lien d'exportation CSV standard (gid=0 pour la première feuille)
    csv_export_link = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&gid=0'

    print("--- Tentative d'importation ---")
    print(f"Lien CSV construit : {csv_export_link}")

    # --- IMPORTATION DU DATAFRAME ---
    try:
        df_clients = pd.read_csv(csv_export_link)

        # Vérification si les colonnes sont fusionnées
        if len(df_clients.columns) == 1 and df_clients.columns[0].count(',') > 0:
            print("\n Avertissement : Les colonnes sont fusionnées. Tentative de correction avec le séparateur ';'.")
            df_clients = pd.read_csv(csv_export_link, sep=';')

        print("\nmportation réussie ! ")


    except Exception as e:
        print(f"\n Erreur lors de l'importation. Assurez-vous que votre document est 'Public sur le web' : {e}")

else:
    print(" Erreur : Impossible d'extraire l'ID du lien Google Sheets. Veuillez vérifier le lien de partage.")

--- Tentative d'importation ---
Lien CSV construit : https://docs.google.com/spreadsheets/d/1SSAlS1aGriDgKKu2AaCGksi4ThEnLBaNNcMklx5dAJU/gviz/tq?tqx=out:csv&gid=0

mportation réussie ! 


In [ ]:
import pandas as pd
import numpy as np


df_dates_initiales = df_clients['date_inscription'].copy()

# 1. Tenter la conversion avec le format ISO (AAAA-MM-JJ)
dates_iso = pd.to_datetime(df_dates_initiales, format='%Y-%m-%d', errors='coerce')

# 2. Tenter la conversion avec le format EU (JJ/MM/AAAA) sur les valeurs qui ont échoué en ISO
# Utilise fillna pour combler les NaT d'ISO avec les dates valides du format EU
dates_eu = pd.to_datetime(df_dates_initiales, format='%d/%m/%Y', errors='coerce')

# 3. Fusionner les deux colonnes (priorité à ISO, sinon EU)
# La colonne finale prendra la date ISO si elle existe, sinon la date EU si elle existe
df_clients['date_inscription_clean'] = dates_iso.fillna(dates_eu)

# --- Vérification de la Récupération ---
nan_avant_fusion = df_clients['date_inscription'].isnull().sum()
nan_apres_fusion = df_clients['date_inscription_clean'].isnull().sum()

print("--- DIAGNOSTIC DE RÉCUPÉRATION DES DATES ---")
print(f"Dates NaT/manquantes avant correction manuelle : {nan_avant_fusion} (Approximatif)")
print(f"Dates NaT/manquantes après correction par double-format : {nan_apres_fusion}")
print(f"Lignes potentiellement sauvées : {nan_avant_fusion - nan_apres_fusion}")

# Remplacer la colonne originale par la colonne nettoyée
df_clients['date_inscription'] = df_clients['date_inscription_clean']
df_clients.drop(columns=['date_inscription_clean'], inplace=True)

--- DIAGNOSTIC DE RÉCUPÉRATION DES DATES ---
Dates NaT/manquantes avant correction manuelle : 36 (Approximatif)
Dates NaT/manquantes après correction par double-format : 364
Lignes potentiellement sauvées : -328


In [ ]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_client         1000 non-null   object        
 1   nom               1000 non-null   object        
 2   genre             770 non-null    object        
 3   pays              798 non-null    object        
 4   email             970 non-null    object        
 5   date_inscription  636 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 47.0+ KB


In [ ]:
df_clients.head()

,id_client,nom,genre,pays,email,date_inscription
0,C0001,Omar,m,Tunisie,omar755@gmail.com,2023-01-21
1,C0002,Fatima,M,France,fatima518@hotmail.com,NaT
2,C0003,Yassine,NaN,Belgique,yassine604@yahoo.com,NaT
3,C0004,Ilyas,F,Tunisie,ilyas345@gmail.com,NaT
4,C0005,Youssef,M,Spain,youssef997@yahoo.com,2021-08-23


In [ ]:
df_clients.isnull().sum()

,0
id_client,0
nom,0
genre,230
pays,202
email,30
date_inscription,364


In [ ]:
import pandas as pd
import numpy as np

# Standardisation de la casse et des espaces, gestion des chaînes 'nan'
df_clients['genre'] = df_clients['genre'].astype(str).str.lower().str.strip().replace('nan', np.nan)
df_clients['pays'] = df_clients['pays'].astype(str).str.capitalize().str.strip().replace('Nan', np.nan)
# Remplacement des NaN par la valeur la plus fréquente (la Mode)
mode_genre = df_clients['genre'].mode()[0]
mode_pays = df_clients['pays'].mode()[0]
df_clients['genre'] = df_clients['genre'].fillna(mode_genre)
df_clients['pays']= df_clients['pays'].fillna(mode_pays)
# Suppression de toutes les lignes où : date est invalide (NaT) OU email est manquant (NaN)
df_clients.dropna(subset=['date_inscription', 'email'], inplace=True)

In [ ]:
df_clients.head()

,id_client,nom,genre,pays,email,date_inscription
0,C0001,Omar,m,Tunisie,omar755@gmail.com,2023-01-21
4,C0005,Youssef,m,Spain,youssef997@yahoo.com,2021-08-23
9,C0010,Nada,f,France,nada323@gmail.com,2022-03-21
10,C0011,Salma,f,Tunisie,salma552@yahoo.com,2022-05-27
11,C0012,Nada,f,Morocco,nada774@gmail.com,2020-11-09


In [ ]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 0 to 998
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_client         614 non-null    object        
 1   nom               614 non-null    object        
 2   genre             614 non-null    object        
 3   pays              614 non-null    object        
 4   email             614 non-null    object        
 5   date_inscription  614 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 33.6+ KB


In [ ]:
import pandas as pd
import numpy as np

# Standardisation de la casse et nettoyage des espaces pour 'genre'
df_clients['genre'] = df_clients['genre'].astype(str).str.lower().str.strip().replace('nan', np.nan)

# Standardisation et nettoyage de 'pays' (première lettre en majuscule)
df_clients['pays'] = df_clients['pays'].astype(str).str.capitalize().str.strip().replace('Nan', np.nan)

print("✅ Standardisation des colonnes 'genre' et 'pays' effectuée. La casse est uniforme et les espaces sont retirés.")

✅ Standardisation des colonnes 'genre' et 'pays' effectuée. La casse est uniforme et les espaces sont retirés.


In [ ]:
df_clients.head()

,id_client,nom,genre,pays,email,date_inscription
0,C0001,Omar,m,Tunisie,omar755@gmail.com,2023-01-21
4,C0005,Youssef,m,Spain,youssef997@yahoo.com,2021-08-23
9,C0010,Nada,f,France,nada323@gmail.com,2022-03-21
10,C0011,Salma,f,Tunisie,salma552@yahoo.com,2022-05-27
11,C0012,Nada,f,Morocco,nada774@gmail.com,2020-11-09


In [ ]:
print("\n--- Vérification des valeurs uniques de 'genre' ---")
print(df_clients['genre'].unique())


--- Vérification des valeurs uniques de 'genre' ---
['m' 'f']


In [ ]:
df_clients.duplicated(keep='first').sum()

np.int64(0)

## **PRODUIT**

In [ ]:
import pandas as pd
import re

# Lien de partage pour 'produits_raw'
share_link_produits = 'https://docs.google.com/spreadsheets/d/1i0xYPCjQ-VJ4-9s3OSuEUaQ_zHn6sLwC/edit?usp=sharing&ouid=111438272798378138165&rtpof=true&sd=true'

# --- TRANSFORMATION AUTOMATIQUE DU LIEN ---
# Extrait l'ID de la feuille de calcul
match = re.search(r'/d/([a-zA-Z0-9_-]+)', share_link_produits)

if match:
    spreadsheet_id = match.group(1)
    # Construction du lien d'exportation CSV (gid=0 pour la première feuille)
    csv_export_link = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&gid=0'

    print("--- Tentative d'importation ---")

    # --- IMPORTATION DU DATAFRAME ---
    try:
        # Tenter la lecture avec le séparateur point-virgule (souvent le cas des exports Sheets)
        df_produits = pd.read_csv(csv_export_link, sep=';')

        # Si le séparateur ';' ne fonctionne pas (colonnes fusionnées), tenter la virgule
        if len(df_produits.columns) < 2: # Une feuille de produit aura au moins 2 colonnes (ID, Nom)
             df_produits = pd.read_csv(csv_export_link, sep=',')

        if len(df_produits.columns) < 2:
             print("Échec : Les colonnes restent fusionnées. Vérifiez le séparateur.")
             exit()

        print("\n Importation de df_produits réussie.")


    except Exception as e:
        print(f"\nErreur lors de l'importation. Assurez-vous que le document est 'Public sur le web' : {e}")

else:
    print("Erreur : Impossible d'extraire l'ID du lien Google Sheets.")

--- Tentative d'importation ---

 Importation de df_produits réussie.


In [ ]:
df_produits.head()

,id_produit,nom_produit,categorie,prix,matiere,origine,poids
0,P0001,Shampoo Bio,Bureau,NaN,Coton bio,Allemagne,"4,42"
1,P0002,Vélo Électrique,Énergie,"7270,8",Cuir végétal,France,"4,98"
2,P0003,Shampoo Bio,Cuisine,NaN,Métal,NaN,NaN
3,P0004,Dentifrice Bio,Bureau,"2755,83",Métal,Portugal,"0,24"
4,P0005,Savon Bio,Cuisine,"5365,66",Bois,France,"0,46"


In [ ]:
df_produits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_produit   1000 non-null   object
 1   nom_produit  1000 non-null   object
 2   categorie    1000 non-null   object
 3   prix         782 non-null    object
 4   matiere      875 non-null    object
 5   origine      737 non-null    object
 6   poids        909 non-null    object
dtypes: object(7)
memory usage: 54.8+ KB


In [ ]:
# --- Standardisation et Imputation de 'matiere' ---

# 1. Standardisation (tout en minuscule)
df_produits['matiere'] = df_produits['matiere'].astype(str).str.lower().str.strip().replace('nan', np.nan)
# 2. Imputation par la Mode
mode_matiere = df_produits['matiere'].mode()[0]
df_produits['matiere'] = df_produits['matiere'].fillna(mode_matiere)

In [ ]:
# --- Standardisation et Imputation de 'origine' ---

# 1. Standardisation (première lettre en majuscule, pour un nom de pays)
df_produits['origine'] = df_produits['origine'].astype(str).str.capitalize().str.strip().replace('Nan', np.nan)
# 2. Imputation par la Mode
mode_origine = df_produits['origine'].mode()[0]
df_produits['origine'] = df_produits['origine'].fillna(mode_origine)

In [ ]:
# Nettoyage de 'prix'
df_produits['prix'] = (
    df_produits['prix']
    .astype(str)
    .str.replace(',', '.', regex=False) # Remplace les virgules par des points
)
df_produits['prix'] = pd.to_numeric(df_produits['prix'], errors='coerce')
# Imputation par la Médiane
median_prix = df_produits['prix'].median()
df_produits['prix'] = df_produits['prix'].fillna(median_prix)
print(f"Colonne 'prix' convertie en float et les NaN imputés par la médiane : {median_prix:.2f}")

Colonne 'prix' convertie en float et les NaN imputés par la médiane : 4115.35


In [ ]:
# Nettoyage de 'poids' (même problème de séparateur)
df_produits['poids'] = (
    df_produits['poids']
    .astype(str)
    .str.replace(',', '.', regex=False) # Remplace les virgules par des points
)
df_produits['poids'] = pd.to_numeric(df_produits['poids'], errors='coerce')

# Imputation par la Médiane
median_poids = df_produits['poids'].median()
df_produits['poids'] = df_produits['poids'].fillna(median_poids)
print(f" Colonne 'poids' convertie en float et les NaN imputés par la médiane : {median_poids:.2f}")

 Colonne 'poids' convertie en float et les NaN imputés par la médiane : 2.49


In [ ]:
df_produits.head()

,id_produit,nom_produit,categorie,prix,matiere,origine,poids
0,P0001,Shampoo Bio,Bureau,4115.355,coton bio,Allemagne,4.42
1,P0002,Vélo Électrique,Énergie,7270.800,cuir végétal,France,4.98
2,P0003,Shampoo Bio,Cuisine,4115.355,métal,France,2.49
3,P0004,Dentifrice Bio,Bureau,2755.830,métal,Portugal,0.24
4,P0005,Savon Bio,Cuisine,5365.660,bois,France,0.46


In [ ]:
df_produits.isnull().sum()

,0
id_produit,0
nom_produit,0
categorie,0
prix,0
matiere,0
origine,0
poids,0


In [ ]:
# Compter les doublons basés sur l'ID produit
doublons_produit = df_produits.duplicated(subset=['id_produit'], keep='first').sum()

if doublons_produit == 0:
    print("\n Aucun doublon critique trouvé dans 'id_produit'.")
else:
    print(f"\n {doublons_produit} doublons trouvés dans 'id_produit'. Nous devons les supprimer.")


 Aucun doublon critique trouvé dans 'id_produit'.


# **VENTES**

In [ ]:
import pandas as pd
import re

# Lien de partage pour 'ventes_raw' (URL fournie par l'utilisateur)
share_link_ventes = 'https://docs.google.com/spreadsheets/d/190nONGYQpUkoT_XacmPxd3zKxo8I2Zgr6LUOlUQK7i8/edit?usp=sharing'

# --- TRANSFORMATION AUTOMATIQUE DU LIEN ---
match = re.search(r'/d/([a-zA-Z0-9_-]+)', share_link_ventes)

if match:
    spreadsheet_id = match.group(1)
    # Construction du lien d'exportation CSV (gid=0 pour la première feuille)
    csv_export_link = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&gid=0'

    print("--- Tentative d'importation de la table des faits ---")

    # --- IMPORTATION DU DATAFRAME ---
    try:
        # Tenter la lecture avec le séparateur point-virgule (courant), puis la virgule
        df_ventes = pd.read_csv(csv_export_link, sep=';')

        # Si la lecture est mauvaise (colonnes fusionnées), tenter avec la virgule
        if len(df_ventes.columns) < 5:
             df_ventes = pd.read_csv(csv_export_link, sep=',')

        if len(df_ventes.columns) < 5:
             print(" Échec : Les colonnes restent fusionnées. Vérifiez le séparateur ou le lien.")
             exit()

        print("\nImportation de df_ventes réussie.")

    except Exception as e:
        print(f"Erreur lors de l'importation. Assurez-vous que le document est 'Public sur le web' : {e}")

else:
    print("Erreur : Impossible d'extraire l'ID du lien Google Sheets.")

--- Tentative d'importation de la table des faits ---

Importation de df_ventes réussie.


In [ ]:
import pandas as pd
import numpy as np

# Suppression des avertissements pour l'exécution
pd.options.mode.chained_assignment = None

print("--- Récupération Maximale des Dates de Vente (Sans Suppression) ---")

# Utiliser la colonne telle qu'elle est après la première importation brute
df_dates_raw = df_ventes['date_vente'].copy()

# 1. Tenter la conversion ISO (Tiret : AAAA-MM-JJ)
# Utilisation de .astype(str) pour gérer les NaN bruts
date_iso_tiret = pd.to_datetime(
    df_dates_raw.astype(str).str.strip(),
    format='%Y-%m-%d',
    errors='coerce'
)

# 2. Tenter la conversion EU (Slash : JJ/MM/AAAA)
date_eu_slash = pd.to_datetime(
    df_dates_raw.astype(str).str.strip(),
    format='%d/%m/%Y',
    errors='coerce'
)

# 3. Tenter la conversion ISO/Mixte (Slash : AAAA/MM/JJ)
date_iso_slash = pd.to_datetime(
    df_dates_raw.astype(str).str.strip(),
    format='%Y/%m/%d',
    errors='coerce'
)

# --- Fusion des Formats ---
# La fusion remplace les NaT d'une tentative par les dates valides des suivantes
date_finale = date_iso_tiret.fillna(date_eu_slash).fillna(date_iso_slash)

# Remplacer la colonne originale par la date nettoyée (conservant les NaT irrécupérables)
df_ventes['date_vente'] = date_finale

# --- Vérification de la Récupération ---
# Le nombre de NaN initiales est supposé être le même que le NaT après conversion
nan_apres = df_ventes['date_vente'].isnull().sum()
print(f"Dates NaT restantes après correction manuelle : {nan_apres}")

# Rétablir les avertissements
pd.options.mode.chained_assignment = 'warn'

--- Récupération Maximale des Dates de Vente (Sans Suppression) ---
Dates NaT restantes après correction manuelle : 100


In [ ]:
df_ventes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_vente    1005 non-null   object        
 1   id_client   1005 non-null   object        
 2   id_produit  1005 non-null   object        
 3   date_vente  905 non-null    datetime64[ns]
 4   quantite    920 non-null    float64       
 5   montant     1005 non-null   object        
 6   CO2_sauve   930 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 55.1+ KB


In [ ]:
df_ventes.head()

,id_vente,id_client,id_produit,date_vente,quantite,montant,CO2_sauve
0,V0001,C0663,P0087,2025-01-22,1.0,6260.57,0.98
1,V0002,C0412,P0459,2025-09-06,NaN,5544.09,NaN
2,V0003,C0643,P0680,2024-12-25,2.0,7714.64,2.54
3,V0004,C0858,P0855,2023-02-06,1.0,7116.85,0.10
4,V0005,C0699,P0641,2025-11-25,3.0,6802.55,1.73


In [ ]:
# Utiliser la colonne telle qu'elle est après la première importation brute
df_dates_raw = df_ventes['date_vente'].copy()

# S'assurer que les valeurs sont traitées comme des chaînes, en enlevant les espaces
dates_nettoyees = df_dates_raw.astype(str).str.strip()

# 1. Tenter la conversion ISO (Tiret : AAAA-MM-JJ)
date_iso_tiret = pd.to_datetime(
    dates_nettoyees,
    format='%Y-%m-%d',
    errors='coerce'
)

# 2. Tenter la conversion EU (Slash : JJ/MM/AAAA)
date_eu_slash = pd.to_datetime(
    dates_nettoyees,
    format='%d/%m/%Y',
    errors='coerce'
)

# 3. Tenter la conversion ISO/Mixte (Slash : AAAA/MM/JJ)
date_iso_slash = pd.to_datetime(
    dates_nettoyees,
    format='%Y/%m/%d',
    errors='coerce'
)

# --- Fusion des Formats ---
# La date finale est standardisée. Les dates non lisibles restent NaT.
date_finale_standard = date_iso_tiret.fillna(date_eu_slash).fillna(date_iso_slash)

# Remplacer la colonne originale
df_ventes['date_vente'] = date_finale_standard


In [ ]:
df_ventes.head()

,id_vente,id_client,id_produit,date_vente,quantite,montant,CO2_sauve
0,V0001,C0663,P0087,2025-01-22,1.0,6260.57,0.98
1,V0002,C0412,P0459,2025-09-06,NaN,5544.09,NaN
2,V0003,C0643,P0680,2024-12-25,2.0,7714.64,2.54
3,V0004,C0858,P0855,2023-02-06,1.0,7116.85,0.10
4,V0005,C0699,P0641,2025-11-25,3.0,6802.55,1.73


In [ ]:
import pandas as pd
import numpy as np


# Remplacement des virgules par des points et conversion en float
df_ventes['montant'] = (
    df_ventes['montant']
    .astype(str)
    .str.replace(',', '.', regex=False)
)

# A. Imputation de 'quantite' (85 NaN)
median_quantite = df_ventes['quantite'].median()
df_ventes['quantite'] = df_ventes['quantite'].fillna(median_quantite)
print(f"Colonne 'quantite' imputée par la médiane : {median_quantite:.2f}")

# B. Imputation de 'CO2_sauve' (75 NaN)
median_co2 = df_ventes['CO2_sauve'].median()
df_ventes['CO2_sauve'] = df_ventes['CO2_sauve'].fillna(median_co2)
print(f" Colonne 'CO2_sauve' imputée par la médiane : {median_co2:.2f}")

# Assurez-vous que df_ventes est l'état actuel après l'imputation de quantite/CO2_sauve
lignes_initiales = len(df_ventes)
nan_a_supprimer = df_ventes['date_vente'].isnull().sum()

print("\n--- 2. Finalisation de 'montant' ---")

# Conversion en numérique (float)
df_ventes['montant'] = pd.to_numeric(df_ventes['montant'], errors='coerce')

# Imputation de 'montant' (c'est ici que les 225 NaN sont traités)
median_montant = df_ventes['montant'].median()
df_ventes['montant'] = df_ventes['montant'].fillna(median_montant)
print(f" Colonne 'montant' convertie en float et imputée par la médiane : {median_montant:.2f}")

# --- 3. Suppression Critique (NaT de Date & Doublons ID) ---
print("\n--- 3. Suppression des Lignes Irrécupérables ---")

# Suppression des doublons ID et des dates invalides (NaT)
df_ventes.drop_duplicates(subset=['id_vente'], keep='first', inplace=True)
df_ventes.dropna(subset=['date_vente'], inplace=True) # Supprime les 100 NaT

lignes_apres_suppression = len(df_ventes)
lignes_supprimees = lignes_initiales - lignes_apres_suppression

print(f" {nan_a_supprimer} lignes ont été supprimées car la date était irrécupérable.")
print(f"Total des lignes supprimées (y compris doublons ID) : {lignes_supprimees}")
print(f"Lignes finales dans Fact_Ventes : {lignes_apres_suppression}")

# Finalisation du DataFrame de Fait
Fact_Ventes = df_ventes.copy()
print("\n La table Fact_Ventes est prête pour l'analyse !")
pd.options.mode.chained_assignment = 'warn'

Colonne 'quantite' imputée par la médiane : 3.00
 Colonne 'CO2_sauve' imputée par la médiane : 2.54

--- 2. Finalisation de 'montant' ---
 Colonne 'montant' convertie en float et imputée par la médiane : 4392.86

--- 3. Suppression des Lignes Irrécupérables ---
 100 lignes ont été supprimées car la date était irrécupérable.
Total des lignes supprimées (y compris doublons ID) : 105
Lignes finales dans Fact_Ventes : 900

 La table Fact_Ventes est prête pour l'analyse !


In [ ]:
import pandas as pd
import numpy as np

# Suppression des avertissements pour l'exécution (si nécessaire)
pd.options.mode.chained_assignment = None

print("--- 2. Suppression des Lignes Irrécupérables ---")

lignes_initiales = len(df_ventes)
nan_a_supprimer = df_ventes['date_vente'].isnull().sum()

# Suppression des doublons ID et des dates invalides (NaT)
df_ventes.drop_duplicates(subset=['id_vente'], keep='first', inplace=True)
df_ventes.dropna(subset=['date_vente'], inplace=True) # Supprime les NaT

lignes_apres_suppression = len(df_ventes)
lignes_supprimees = lignes_initiales - lignes_apres_suppression

print(f" {nan_a_supprimer} lignes ont été supprimées car la date était irrécupérable.")
print(f"Total des lignes supprimées (y compris doublons ID) : {lignes_supprimees}")
print(f"Lignes finales dans Fact_Ventes : {lignes_apres_suppression}")

# Finalisation du DataFrame de Fait
df_ventes = df_ventes.copy()
print("\n La table df_ventes est prête pour l'analyse !")
pd.options.mode.chained_assignment = 'warn'

--- 2. Suppression des Lignes Irrécupérables ---
 0 lignes ont été supprimées car la date était irrécupérable.
Total des lignes supprimées (y compris doublons ID) : 0
Lignes finales dans Fact_Ventes : 900

 La table df_ventes est prête pour l'analyse !


In [ ]:
df_ventes.head()

,id_vente,id_client,id_produit,date_vente,quantite,montant,CO2_sauve
0,V0001,C0663,P0087,2025-01-22,1.0,6260.57,0.980
1,V0002,C0412,P0459,2025-09-06,3.0,5544.09,2.535
2,V0003,C0643,P0680,2024-12-25,2.0,7714.64,2.540
3,V0004,C0858,P0855,2023-02-06,1.0,7116.85,0.100
4,V0005,C0699,P0641,2025-11-25,3.0,6802.55,1.730


In [ ]:
df_ventes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_vente    900 non-null    object        
 1   id_client   900 non-null    object        
 2   id_produit  900 non-null    object        
 3   date_vente  900 non-null    datetime64[ns]
 4   quantite    900 non-null    float64       
 5   montant     900 non-null    float64       
 6   CO2_sauve   900 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 56.2+ KB


# **LIVRAISON**

In [ ]:
from google.colab import files

# Ouvre une boîte de dialogue pour sélectionner le fichier sur votre ordinateur
uploaded = files.upload()

# Le nom de fichier est la clé du dictionnaire 'uploaded'
file_name = list(uploaded.keys())[0]

print(f" Fichier '{file_name}' téléchargé dans l'environnement Colab.")

Saving livraisons_raw.json to livraisons_raw (4).json
 Fichier 'livraisons_raw (4).json' téléchargé dans l'environnement Colab.


In [ ]:
import pandas as pd

# 2. Lecture du fichier JSON
df_livraisons = pd.read_json(file_name)

print("\n Importation de df_livraisons réussie.")
print("--- Diagnostic initial (df_livraisons.info()) ---")
df_livraisons.info()


 Importation de df_livraisons réussie.
--- Diagnostic initial (df_livraisons.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_livraison    1000 non-null   object
 1   id_vente        1000 non-null   object
 2   transporteur    1000 non-null   object
 3   ville_depart    1000 non-null   object
 4   ville_arrivee   1000 non-null   object
 5   delai           757 non-null    object
 6   cout_transport  1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [ ]:
df_livraisons.head()

,id_livraison,id_vente,transporteur,ville_depart,ville_arrivee,delai,cout_transport
0,L0001,V0880,DHL,Marakech,Marrakech,4 jours,170.3
1,L0002,V0878,Poste Maroc,Marrakech,Tanger,10 jours,179.29
2,L0003,V0823,Chronopost,Oujda,Rabat,trois jours,19.73
3,L0004,V0903,Chronopost,Agadir,Marrakech,3,
4,L0005,V0397,,Marakech,Fès,1 jours,154MAD


In [ ]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

print("--- 1. Standardisation des Vides (Pré-Imputation) ---")
# Application de la standardisation à toutes les colonnes object
object_cols = df_livraisons.select_dtypes(include=['object']).columns
for col in object_cols:
    df_livraisons[col] = df_livraisons[col].astype(str).str.strip()
    df_livraisons[col].replace('', np.nan, inplace=True)

print(" Conversion des chaînes vides en NaN terminée pour toute la table.")

# --- 2. Imputation et Finalisation ---
print("\n--- 2. Imputation et Finalisation de Dim_Livraison ---")

# A. Imputation de transporteur (Mode)
nan_transporteur = df_livraisons['transporteur'].isnull().sum()
if nan_transporteur > 0:
    mode_transporteur = df_livraisons['transporteur'].mode()[0]
    df_livraisons['transporteur'] = df_livraisons['transporteur'].fillna(mode_transporteur)
    print(f" 'transporteur' : {nan_transporteur} NaN imputés par la Mode ({mode_transporteur}).")

# B. Imputation et Conversion de cout_transport (Médiane)
df_livraisons['cout_transport'] = df_livraisons['cout_transport'].astype(str).str.replace(',', '.', regex=False)
df_livraisons['cout_transport'] = pd.to_numeric(df_livraisons['cout_transport'], errors='coerce')
nan_cout_apres = df_livraisons['cout_transport'].isnull().sum()
if nan_cout_apres > 0:
    median_cout = df_livraisons['cout_transport'].median()
    df_livraisons['cout_transport'] = df_livraisons['cout_transport'].fillna(median_cout)
    print(f" 'cout_transport' : {nan_cout_apres} NaN imputés par la médiane ({median_cout:.2f}).")

# C. Imputation de delai (Médiane)
df_livraisons['delai'] = pd.to_numeric(df_livraisons['delai'], errors='coerce')
nan_delai_apres = df_livraisons['delai'].isnull().sum()
if nan_delai_apres > 0:
    median_delai = df_livraisons['delai'].median()
    df_livraisons['delai'] = df_livraisons['delai'].fillna(median_delai)
    print(f"'delai' : {nan_delai_apres} NaN imputés par la médiane ({median_delai:.2f}).")

# D. Création de la Dimension Finale (Suppression des doublons)
Dim_Livraison = df_livraisons.drop_duplicates(subset=['id_livraison'], keep='first').copy()
doublons_supprimes = len(df_livraisons) - len(Dim_Livraison)

print(f"\n Dim_Livraison est finalisée.")
print(f"Nombre final de lignes : {len(Dim_Livraison)}")

pd.options.mode.chained_assignment = 'warn'

--- 1. Standardisation des Vides (Pré-Imputation) ---
 Conversion des chaînes vides en NaN terminée pour toute la table.

--- 2. Imputation et Finalisation de Dim_Livraison ---
 'transporteur' : 149 NaN imputés par la Mode (AMANA).
 'cout_transport' : 475 NaN imputés par la médiane (102.69).
'delai' : 740 NaN imputés par la médiane (5.00).

 Dim_Livraison est finalisée.
Nombre final de lignes : 1000


/tmp/ipython-input-3897300630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livraisons[col].replace('', np.nan, inplace=True)
/tmp/ipython-input-3897300630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [ ]:
df_livraisons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_livraison    1000 non-null   object 
 1   id_vente        1000 non-null   object 
 2   transporteur    1000 non-null   object 
 3   ville_depart    1000 non-null   object 
 4   ville_arrivee   1000 non-null   object 
 5   delai           1000 non-null   float64
 6   cout_transport  1000 non-null   float64
dtypes: float64(2), object(5)
memory usage: 54.8+ KB


In [ ]:
# S'assurer que les clés de jointure sont propres (important pour l'intégrité)
df_ventes['id_vente'] = df_ventes['id_vente'].astype(str).str.upper().str.strip()
df_livraisons['id_vente'] = df_livraisons['id_vente'].astype(str).str.upper().str.strip()

print(" Clés de jointure standardisées.")

 Clés de jointure standardisées.


In [ ]:
import pandas as pd
import numpy as np

print("--- Exécution de la Jointure sur df_ventes ---")

# 1. Sélectionner uniquement les deux colonnes nécessaires de Dim_Livraison
# Renommez la table Dim_Livraison pour vous assurer que c'est la bonne source.
dim_keys = Dim_Livraison[['id_vente', 'id_livraison']].copy()

# 2. Joindre df_ventes avec les clés de livraison
# La destination DOIT être df_ventes.
df_ventes = pd.merge(
    df_ventes,
    dim_keys,
    on='id_vente',
    how='left'
)

# 3. Vérification du nombre de valeurs manquantes dans df_ventes
# Nous vérifions maintenant la colonne nouvellement ajoutée dans df_ventes
ventes_sans_livraison = df_ventes['id_livraison'].isnull().sum()

print(" Jointure terminée. Nouvelle colonne 'id_livraison' ajoutée à df_ventes.")
print(f" Nombre de ventes sans identifiant de livraison : {ventes_sans_livraison}")

--- Exécution de la Jointure sur df_ventes ---
 Jointure terminée. Nouvelle colonne 'id_livraison' ajoutée à df_ventes.
 Nombre de ventes sans identifiant de livraison : 349


In [ ]:
df_ventes.head()


,id_vente,id_client,id_produit,date_vente,quantite,montant,CO2_sauve,id_livraison
0,V0001,C0663,P0087,2025-01-22,1.0,6260.57,0.980,L0887
1,V0002,C0412,P0459,2025-09-06,3.0,5544.09,2.535,L0222
2,V0002,C0412,P0459,2025-09-06,3.0,5544.09,2.535,L0484
3,V0003,C0643,P0680,2024-12-25,2.0,7714.64,2.540,NaN
4,V0004,C0858,P0855,2023-02-06,1.0,7116.85,0.100,L0164


In [ ]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
df_ventes = df_ventes # Utiliser le DataFrame de la table de fait après jointure

print("--- Imputation Définitive des Clés 'id_livraison' Manquantes ---")

# Définir la clé Surrogate (Clé pour "Inconnu")
CLE_INCONNUE = 'L0000'

# 1. Compter les NaN après la jointure
nan_a_imputer = df_ventes['id_livraison'].isnull().sum()

if nan_a_imputer > 0:
    # 2. Imputer toutes les valeurs manquantes avec la clé Inconnue (L0000)
    df_ventes['id_livraison'].fillna(CLE_INCONNUE, inplace=True)

    # 3. Vérifier et ajouter la clé Inconnue dans la table df_livraisons
    # Ceci garantit que toutes les clés étrangères dans df_ventes ont un correspondant dans la Dimension.
    if CLE_INCONNUE not in df_livraisons['id_livraison'].values:
        print(f"Ajout de la ligne '{CLE_INCONNUE}' dans df_livraisons.")

        nouvelle_ligne_dim = {
            'id_livraison': CLE_INCONNUE,
            'id_vente': 'N/A',
            'transporteur': 'INCONNU',
            'ville_depart': 'INCONNU',
            'ville_arrivee': 'INCONNU',
            'delai': 0.0,
            'cout_transport': 0.0
        }
        # Utilisation de pd.concat car la ligne est une nouvelle dimension
        df_livraisons = pd.concat([df_livraisons, pd.DataFrame([nouvelle_ligne_dim])], ignore_index=True)

    print(f" {nan_a_imputer} clés 'id_livraison' imputées par la clé de référence '{CLE_INCONNUE}'.")

else:
    print(" Aucune clé 'id_livraison' manquante trouvée dans df_ventes.")

print(f"Nombre final de lignes dans df_livraisons (incluant Inconnu) : {len(df_livraisons)}")

--- Imputation Définitive des Clés 'id_livraison' Manquantes ---
Ajout de la ligne 'L0000' dans df_livraisons.
 349 clés 'id_livraison' imputées par la clé de référence 'L0000'.
Nombre final de lignes dans df_livraisons (incluant Inconnu) : 1001


/tmp/ipython-input-407424276.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ventes['id_livraison'].fillna(CLE_INCONNUE, inplace=True)


In [ ]:
import pandas as pd
import numpy as np

# Copier la colonne pour travailler dessus sans impacter la table finale immédiatement
colonne_test = df_livraisons['transporteur'].copy()

print("--- Vérification Agressive de toutes les formes de valeurs manquantes ---")

# 1. Standardisation: retirer espaces et mettre en majuscules (pour capturer 'nan' vs 'NaN')
colonne_test = colonne_test.astype(str).str.strip().str.upper()

# 2. Remplacement agressif : toutes les chaînes vides ou de type "null"
# On inclut 'INCONNU' au cas où vous vouliez compter les valeurs imputées comme manquantes.
colonne_test.replace(['', 'NAN', 'NONE', 'N/A', 'NA', 'INCONNU'], np.nan, inplace=True)

# 3. Compter les véritables NaN
nan_agressif = colonne_test.isnull().sum()

print(f"Nombre de valeurs techniquement manquantes dans 'transporteur' : {nan_agressif}")

--- Vérification Agressive de toutes les formes de valeurs manquantes ---
Nombre de valeurs techniquement manquantes dans 'transporteur' : 131


In [ ]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

print("--- Correction et Imputation Définitive de 'transporteur' ---")

# 1. Standardisation et conversion en NaN (pour capturer les 131 erreurs)
df_livraisons['transporteur'] = df_livraisons['transporteur'].astype(str).str.strip().str.upper()
df_livraisons['transporteur'].replace(['', 'NAN', 'NONE', 'N/A', 'NA'], np.nan, inplace=True)

nan_actuel = df_livraisons['transporteur'].isnull().sum()
print(f"1. Nombre de NaN capturés et prêts à être imputés : {nan_actuel}")


# 2. Imputation Finale par la Mode
if nan_actuel > 0:
    # Recalculer la mode sur les valeurs restantes valides
    mode_transporteur = df_livraisons['transporteur'].mode()[0]
    df_livraisons['transporteur'].fillna(mode_transporteur, inplace=True)

    # 3. Vérification finale
    nan_final = df_livraisons['transporteur'].isnull().sum()

    if nan_final == 0:
        print(f" 'transporteur' est maintenant COMPLET. {nan_actuel} NaN imputés par la Mode ({mode_transporteur}).")
    else:
        print(f" Erreur: Il reste {nan_final} NaN après imputation.")
else:
    print(" Aucune valeur manquante à imputer.")

print(f"Nombre final de lignes dans df_livraisons : {len(df_livraisons)}")

--- Correction et Imputation Définitive de 'transporteur' ---
1. Nombre de NaN capturés et prêts à être imputés : 130
 'transporteur' est maintenant COMPLET. 130 NaN imputés par la Mode (AMANA).
Nombre final de lignes dans df_livraisons : 1001


/tmp/ipython-input-2349624137.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_livraisons['transporteur'].replace(['', 'NAN', 'NONE', 'N/A', 'NA'], np.nan, inplace=True)


In [ ]:
import pandas as pd
from google.colab import files

print("--- Exportation des Tables Finales ---")

# 1. Définir les DataFrames et les noms de fichiers
tables_to_export = {
    'Dim_Client': df_clients,
    'Dim_Produit': df_produits,
    'Dim_Livraison': df_livraisons,
    'Fact_Ventes': df_ventes  # Assurez-vous que Fact_Ventes est dans son état final (après intégrité référentielle)
}

# 2. Exporter chaque table en fichier CSV
for df_name, df_data in tables_to_export.items():

    # Créer le nom du fichier CSV
    file_name = f"DW_FINAL_{df_name}.csv"

    # Exporter le DataFrame en CSV
    # index=False pour ne pas inclure l'index de Pandas dans le fichier final
    df_data.to_csv(file_name, index=False, encoding='utf-8')

    print(f"Fichier '{file_name}' créé avec {len(df_data)} lignes.")

print("\n Tous les fichiers ont été créés et sont prêts au téléchargement.")

# 3. Ouvrir la boîte de dialogue de téléchargement de Colab
# Le résultat affichera des liens de téléchargement que vous devrez cliquer.
try:
    files.download('DW_FINAL_Dim_Client.csv')
    files.download('DW_FINAL_Dim_Produit.csv')
    files.download('DW_FINAL_Dim_Livraison.csv')
    files.download('DW_FINAL_Fact_Ventes.csv')
except NameError:
    print(" Les fichiers ne peuvent pas être téléchargés automatiquement. Vous pouvez les trouver dans l'Explorateur de Fichiers de Colab.")

--- Exportation des Tables Finales ---
Fichier 'DW_FINAL_Dim_Client.csv' créé avec 614 lignes.
Fichier 'DW_FINAL_Dim_Produit.csv' créé avec 1000 lignes.
Fichier 'DW_FINAL_Dim_Livraison.csv' créé avec 1001 lignes.
Fichier 'DW_FINAL_Fact_Ventes.csv' créé avec 1231 lignes.

 Tous les fichiers ont été créés et sont prêts au téléchargement.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>